### 베이스 라인 모델링
세 단계에 걸쳐 모델링을 진행한다.

- 1. 데이터 준비 과정에서 이미지 변환기로 데이터를 증강
- 2. 사전 훈련된 모델을 이용해 전이학습 진행
- 3. 모델 훈련과 성능 검증을 동시에 진행하면서 훈련을 반복

In [1]:
# 시드값 고정, GPU 장비 설정
import torch
import random
import numpy as np
import os

# 시드값 고정
seed = 50
os.environ['PYTHONHASHSPEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enable = False

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### 데이터 불러오기

데이터를 불러와서 train, validation set으로 분할한다

In [3]:
import pandas as pd

train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
sub = pd.read_csv('./data/sample_submission.csv')

In [4]:
from sklearn.model_selection import train_test_split

# test_size = train set : validation set의 비율을 정한다
# stratify = 데이터 클래스 분포 비율을 맞춰주는 옵션 
train, validation = train_test_split(train, test_size=0.1, stratify=train[['healthy', 'multiple_diseases', 'rust', 'scab']], random_state=50)

### 데이터 셋 클래스 정의하기

파이토치로 신경망 모델을 구축하려면 데이터셋도 일정한 형식에 맞게 정의해줘야한다.

파이토치에서 제공하는 Dataset클래스를 활용해서 데이터셋 객체를 만들 수 있다. Dataset은 추상클래스로, Dataset을 상속받은 특수 메소드인 __len__과 __getitem__을 재정의(오버라이딩) 해야한다.

- __init__() : imageDataset 클래스의 초기화 메소드
    + df : Dataframe 객체로 train 혹은 valid를 df 파라미터에 전달
    + img_dir : 이미지 데이터를 포함하는 경로
    + transform : 이미지 변환기. 이미지 셋을 만들 때 기본적인 전처리를 하기 위해 변환기를 넘겨줌
- __len__() : 데이터셋 크기를 반환
- __getitem__() : 인덱스를 전달받아 인덱스에 해당하는 데이터를 반환

In [5]:
import cv2
from torch.utils.data import Dataset # 데이터 생성을 위한 클래스
import numpy as np

class ImageDataset(Dataset):
    # 변수 초기화
    def __init__(self, df, img_dir = "./", transform=None, is_test=False):
        super().__init__() # 상속받은 Datatset의 __init__()메소드 호출
        # 전달받은 인수 저장
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test
        
    # 데이터셋 크기 반환 메소드
    def __len__(self):
        return len(self.df)
    
    # 인덱스에 해당하는 데이터 반환 메소드
    def __getitem__(self, idx):
        img_id = self.df.iloc[idx, 0] # image ID
        img_path = self.img_dir + img_id + '.jpg' # 이미지 파일경로
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform is not None:
            image = self.transform(image = image)['image'] 
        # 테스트 데이터면 이미지 데이터만 반환, 그렇지 않으면 타겟값도 반환
        if self.is_test:
            return image
        else:
            label = np.argmax(self.df.iloc[idx, 1:5]) # 원핫 인코딩된 숫자들 중에 1만 뽑아내기 위한 값 : 라벨값
            return image, label # 훈련/검증용일때

### 데이터 증강용 이미지 변환기 정의(Train set)

In [6]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

transform_train = A.Compose([
    A.Resize(450, 650), # 이미지 크기 조절
    A.RandomBrightnessContrast(brightness_limit=0.2, # 밝기 대비 조절. brightness_limit : 밝기 조절값을 설정. 0.2로 지정했기에 -0.2~0.2의 범위를 가지며 전체범위는 -1~1까지이다.
                               contrast_limit=0.2, p=0.3), # 1에 가까울수록 밝다. contrast_limit : 이미지 대비 조절값을 설정. 동작방식은 brightness_limit와 동일
                                                           # p : 적용 확률 설정. 0.3을 설정했기에 30%의 확률로 이미지에 변환기를 적용한다.
    A.VerticalFlip(p=0.2), # 상하 반전
    A.HorizontalFlip(p=0.5), # 좌우 반전
    A.ShiftScaleRotate( # 이동, 스케일링, 회전 변환
        shift_limit=0.1, 
        scale_limit=0.2,
        rotate_limit=30, p=0.3),
    A.OneOf([A.Emboss(p = 1), # 양각화, 날카로움, 블러 효과
            A.Sharpen(p = 1), 
            A.Blur(p = 1)], p = 0.3),
    A.PiecewiseAffine(p = 0.3), # 어파인 변환
    A.Normalize(), # 정규화 변환
    ToTensorV2() # 텐서화
])

### 데이터 증강용 이미지 변환기 정의(Test)

In [7]:
# 검증 및 테스트 데이터용 변환기
transform_test = A.Compose([
    A.Resize(450, 650), # 이미지 크기 조절 
    A.Normalize(),      # 정규화 변환
    ToTensorV2()        # 텐서로 변환(torch의 경우 텐서 객체만 취급하기 때문에 tensor 객체로 변환하는 작업이 필요하다.)
])

### 이미지 데이터셋 변환하기

In [8]:
img_dir = './data/images/'

train_set = ImageDataset(train, img_dir=img_dir, transform=transform_train) # train set 증강
test_set = ImageDataset(test, img_dir=img_dir, transform=transform_test) # test set 증강

### 훈련 시간 단축을 위한 멀티 프로세싱

In [9]:
def seed_worker(worker_id): # 시드값 고정 함수
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    
g = torch.Generator()
g.manual_seed(0)

In [10]:
from torch.utils.data import DataLoader

batch_size = 4
#                         이미지 데이터셋, batch size : 한번에 물러오는 데이터 크기, shuffle : 데이터 섞는 여부
loader_train = DataLoader(train_set, batch_size = batch_size, shuffle = True, worker_init_fn = seed_worker, generator = g, num_workers = 2)
loader_valid = DataLoader(test_set, batch_size = batch_size, shuffle = False, worker_init_fn = seed_worker, generator = g, num_workers = 2)

### 모델링
사전에 훈련된 모델을 전이 학습시키는 방식을 사용하여 모델링한다.

- 사전 훈련 모델 : 이미 한 분야에서 훈련을 마친 모델을 의미.
- 전이 학습 : 사전 훈련 모델을 유사한 다른 영역에서 재훈련 시키는 기법.

파이토치로 사전 훈련 모델을 이용하는 방법
1. torchvision.models 모듈 이용
    + 기본적인 사전 훈련 모델을 제공한다. 제공하는 모델이 많지 않다는 단점이 있다.
2. pretrainedmodels 모듈 이용
    + pretrainedmodels도 사전 훈련 모델을 제공한다.

In [11]:
# EfficientNet 모델 생성
from efficientnet_pytorch import EfficientNet
# EfficientNet의 경우 b0~b7까지의 모델이 존재하는데, 숫자가 높을수록 성능이 좋다고 알려져 있다.
model = EfficientNet.from_pretrained('efficientnet-b7', num_classes=4) # num_calsses : 최종 출력값 개수를 뜻한다. 각 문제에 맞게 타겟값의 수로 설정하면 되겠다.
model = model.to(device)

Loaded pretrained weights for efficientnet-b7


### 모델 훈련 및 성능 검증
손실함수와 옵티마이저를 설정한다.

In [12]:
import torch.nn as nn # 신경망 모듈

# loss function
criterion = nn.CrossEntropyLoss()

# optimizer : AdamW는 adam optimizer에 가중치 감쇠(가중치를 작게 조절)를 추가해서 일반화 성능이 우수하다.
optimizer = torch.optim.AdamW(model.parameters(), lr = 0.00006, weight_decay=0.0001)

In [13]:
# 모델 훈련 및 성능 검증
from sklearn.metrics import roc_auc_score # roc, auc 점수 계산 함수
from tqdm.notebook import tqdm

epochs = 5 # 훈련 횟수

for epoch in range(epochs):
    model.train() # 모델을 훈련 상태로 설정
    epoch_train_loss = 0 # 각 에포크 별 loss값 초기화(훈련 데이터용)
    
    # 훈련 횟수 만큼 반복
    for images, labels in tqdm(loader_train):
        images = images.to(device)
        labels = labels.to(device)
        
        # 옵티마이저 내의 기울기 값 초기화
        optimizer.zero_grad()
        
        # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        
        # 손실함수를 활용해 outputs와 labels의 loss값 계산
        loss = criterion(outputs, labels)
        
        # 현재 배치에서의 손실 추가
        epoch_train_loss += loss.item()
        loss.backward() # 역전파 수행
        optimizer.step() # 가중치 갱신
    
    # 훈련 데이터 손실값 출력
    print(f"epoch [{epoch+1}/{epochs}] - 훈련 데이터 손실값 : {epoch_train_loss/len(loader_train):.4f}")
    
    # 검증 실시
    model.eval()          # 모델을 평가 상태로 설정
    epoch_valid_loss = 0  # 에폭별 손실값 초기화(검증 데이터용)
    preds_list = []       # 예측 확률값 저장용 리스트 초기화
    true_onehot_list = [] # 실제 타겟값 저장용 리스트 초기화
    
    with torch.no_grad(): # 기울기 계산 비활성화
        # 미니 배치 단위로 검증
        for images, labels in loader_valid:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            epoch_valid_loss += loss.item()
            
            preds = torch.softmax(outputs.cpu(), dim=1).numpy() # 예측 확률값
            # 실제값 (원핫 인코딩 방식)
            true_onehot = torch.eye(4)[labels].cpu().numpy()
            # 예측확률값과 실제값 저장
            preds_list.extend(preds)
            true_onehot_list.extend(true_onehot)
    print(f"epoch [{epoch+1}/{epochs}] - 검증 데이터 손실값 : {epoch_valid_loss/len(loader_valid):.4f} / 검증 데이터 ROC AUC : {roc_auc_score(true_onehot_list, preds_list):.4f}")

  0%|          | 0/410 [00:00<?, ?it/s]

In [ ]:
()